Галайда ГригорьевС

In [ ]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to your dataset on Google Drive
dataset_path = '/content/drive/My Drive/датасет1'

# Augmentations to apply
seq_gaussian = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 1.0))  # blur images with a sigma of 0 to 1.0
])

seq_affine = iaa.Sequential([
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})
])

seq_rotate = iaa.Sequential([
    iaa.Rotate((-45, 45))  # rotate images by -45 to 45 degrees
])

# Classes to process
classes = ['мыльница', 'доска']

# Function to load and preprocess images
def load_and_preprocess_images(class_path):
    image_files = [f for f in os.listdir(class_path) if f.endswith('.jpg') or f.endswith('.png')]
    images = []
    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        image = cv2.resize(image, (224, 224))  # Resize to 224x224
        images.append(image)
    return images

# Function to save images to Google Drive using cv2
def save_images_to_drive(images, path, prefix):
    if not os.path.exists(path):
        os.makedirs(path)
    for i, image in enumerate(images):
        image_path = os.path.join(path, f'{prefix}_{i}.jpg')  # Save as JPEG
        cv2.imwrite(image_path, image)

# Process each class
for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    augmented_path = os.path.join('/content/drive/My Drive/dataset', class_name)

    # Load and preprocess images
    images = load_and_preprocess_images(class_path)

    # Apply augmentations
    augmented_images_gaussian = seq_gaussian.augment_images(images)
    augmented_images_affine = seq_affine.augment_images(images)
    augmented_images_rotate = seq_rotate.augment_images(images)

    # Save augmented images
    save_images_to_drive(augmented_images_gaussian, augmented_path, 'gaussian')
    save_images_to_drive(augmented_images_affine, augmented_path, 'affine')
    save_images_to_drive(augmented_images_rotate, augmented_path, 'rotate')

# Combine original and augmented images
# This part is not necessary for saving the augmented images, as they are already saved separately.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix

# Указываем путь к папке с датасетом
dataset_path = '/content/drive/My Drive/dataset'

# Предобработка изображений
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Загружаем датасет с предобработкой
dataset = datasets.ImageFolder(dataset_path, transform=preprocess)

# Разделяем датасет на обучающую, валидационную и тестовую выборки
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Создаем загрузчики данных
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Загружаем предобученную модель ResNet18
model = models.resnet18(pretrained=True)

# Заменяем последний слой для бинарной классификации (2 класса)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
# model.fc = nn.Sequential(nn.Linear(num_ftrs, 2), nn.Softmax(dim=1))

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
for epoch in range(10):  # Количество эпох
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Выводим статистику после каждой эпохи
        print(f'Epoch {epoch+1}/10, Loss: {loss.item()}, Val Accuracy: {100 * correct / total}%')

print('Finished Training')

# # Тестируем модель на тестовой выборке
# model.eval()
# with torch.no_grad():
#     correct = 0
#     total = 0
#     predictions = []
#     true_labels = []
#     for inputs, labels in test_loader:
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         predictions.extend(predicted.tolist())
#         true_labels.extend(labels.tolist())

# # Выводим статистику на тестовой выборке
# print(f'Test Accuracy: {100 * correct / total}%')

# # Построение матрицы ошибок
# conf_matrix = confusion_matrix(true_labels, predictions)
# print("Confusion Matrix:")
# print(conf_matrix)

# Сохраняем модель
# torch.save(model.state_dict(), 'model.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.681601881980896, Val Accuracy: 52.68817204301075%
Epoch 1/10, Loss: 0.6095734238624573, Val Accuracy: 59.85663082437276%
Epoch 1/10, Loss: 0.7091283798217773, Val Accuracy: 79.92831541218638%
Epoch 1/10, Loss: 0.4813155233860016, Val Accuracy: 88.17204301075269%
Epoch 1/10, Loss: 0.4473612904548645, Val Accuracy: 93.54838709677419%
Epoch 1/10, Loss: 0.23659095168113708, Val Accuracy: 95.6989247311828%
Epoch 1/10, Loss: 0.30357226729393005, Val Accuracy: 97.4910394265233%
Epoch 1/10, Loss: 0.1998952329158783, Val Accuracy: 98.9247311827957%


KeyboardInterrupt: 

In [31]:
# Тестируем модель на тестовой выборке
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    predictions = []
    true_labels = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predictions.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

# Выводим статистику на тестовой выборке
print(f'Test Accuracy: {100 * correct / total}%')

# Построение матрицы ошибок
conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Test Accuracy: 99.2831541218638%
Confusion Matrix:
[[127   2]
 [  0 150]]


In [32]:
# Сохраняем модель
torch.save(model.state_dict(), 'model.pth')
model_save_path = '/content/drive/My Drive/models'

# Сохранение модели
model_name = 'my_model2.pth'
torch.save(model.state_dict(), model_save_path + '/' + model_name)

In [33]:
# dataset_pth = '/content/drive/My Drive/Check_photos'
dataset_pth = '/content/drive/My Drive/датасет1'

dst = datasets.ImageFolder(dataset_pth, transform=preprocess)
test_loader = DataLoader(dst, batch_size=32, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

# Тестируем модель на тестовой выборке
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    predictions = []
    true_labels = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predictions.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

# Выводим статистику на тестовой выборке
print(f'Test Accuracy: {100 * correct / total}%')

# Построение матрицы ошибок
conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Test Accuracy: 98.87096774193549%
Confusion Matrix:
[[304   6]
 [  1 309]]
